In [1]:
import unittest
from unittest.mock import MagicMock, patch
import os
import sys
import pandas as pd

# Import customised modules

# Define path
sys.path.append(os.path.abspath(os.path.join('..')))

## Import relevant modules
from scripts._01_2_merge_csvs import combine_csvs

class TestCombineCSVs(unittest.TestCase):

    @patch('__main__.os.listdir')
    @patch('__main__.os.path.join')
    @patch('__main__.pd.read_csv')
    @patch('__main__.pd.concat')
    @patch('__main__.os.path.relpath')
    @patch('builtins.print')
    def test_combine_csvs_success(self, mock_print, mock_relpath, mock_concat, mock_read_csv, mock_join, mock_listdir):
        # Mock setup
        csv_dir = '/fake/csv/dir'
        data_dir = '/fake/data/dir'
        mock_listdir.return_value = ['channel1.csv', 'channel2.csv', 'telegram_data.csv', 'misc.txt']
        mock_join.side_effect = lambda *args: '/'.join(args) # Simulate joining paths
        mock_read_csv.return_value = MagicMock(spec=pd.DataFrame) # Mock a DataFrame
        mock_concat.return_value = MagicMock(spec=pd.DataFrame) # Mock the combined DataFrame
        mock_relpath.return_value = '../fake/data/dir/telegram_data.csv'

        # Call the function
        combine_csvs(csv_dir, data_dir)

        # Assertions
        mock_listdir.assert_called_once_with(csv_dir)
        # Check if read_csv was called for the correct files
        mock_read_csv.assert_any_call('/fake/csv/dir/channel1.csv')
        mock_read_csv.assert_any_call('/fake/csv/dir/channel2.csv')
        self.assertEqual(mock_read_csv.call_count, 2) # Ensure it wasn't called for excluded files

        # Check if concat was called with the list of DataFrames
        mock_concat.assert_called_once()
        self.assertEqual(len(mock_concat.call_args[0][0]), 2) # Ensure two DataFrames were concatenated

        # Check if to_csv was called on the combined DataFrame with the correct path
        mock_combined_df = mock_concat.return_value
        mock_combined_df.to_csv.assert_called_once_with('/fake/data/dir/telegram_data.csv', index=False)

        mock_relpath.assert_called_once_with('/fake/data/dir/telegram_data.csv', os.getcwd())
        mock_print.assert_called_once_with("✓ Combined 2 CSVs into ../fake/data/dir/telegram_data.csv")

    @patch('__main__.os.listdir')
    @patch('__main__.os.path.join')
    @patch('__main__.pd.read_csv')
    @patch('__main__.pd.concat')
    @patch('__main__.os.path.relpath')
    @patch('builtins.print')
    def test_combine_csvs_no_csvs(self, mock_print, mock_relpath, mock_concat, mock_read_csv, mock_join, mock_listdir):
        # Mock setup when no relevant CSVs are found
        csv_dir = '/fake/csv/dir'
        data_dir = '/fake/data/dir'
        mock_listdir.return_value = ['telegram_data.csv', 'misc.txt'] # No relevant CSVs
        mock_join.side_effect = lambda *args: '/'.join(args) # Simulate joining paths
        mock_concat.return_value = MagicMock(spec=pd.DataFrame) # Mock the combined DataFrame
        mock_relpath.return_value = '../fake/data/dir/telegram_data.csv'


        # Call the function
        combine_csvs(csv_dir, data_dir)

        # Assertions
        mock_listdir.assert_called_once_with(csv_dir)
        mock_read_csv.assert_not_called() # No CSVs to read
        mock_concat.assert_called_once()
        self.assertEqual(len(mock_concat.call_args[0][0]), 0) # Ensure an empty list was concatenated

        # Check if to_csv was called on the combined DataFrame with the correct path
        mock_combined_df = mock_concat.return_value
        mock_combined_df.to_csv.assert_called_once_with('/fake/data/dir/telegram_data.csv', index=False)

        mock_relpath.assert_called_once_with('/fake/data/dir/telegram_data.csv', os.getcwd())
        mock_print.assert_called_once_with("✓ Combined 0 CSVs into ../fake/data/dir/telegram_data.csv")


# Run the test
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.012s

OK
